# Wine - By the Numbers - Location Information

#### Note
* Evaluate Data Set used in Wine by the numbers to come up with location information

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint
import json
from ipywidgets.embed import embed_minimal_html
from sqlalchemy import create_engine

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Connect to Database (Pre-Work Required)
* Complete all work in Wine_By_Number_ETL

In [3]:
rds_connection_string = "postgres:postgres@localhost:5432/Wine_Analysis"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
# Confirm tables
engine.table_names()

['wine_ratings', 'location_info']

### Load Process

In [5]:
pd.read_sql_query('select * from wine_ratings', con=engine).head()

,country,description,points,price,province,region_1,taster_name,title,variety,winery,wine_year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,None,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


* Loaded distinct country and province for analysis

In [9]:
loc_info_df = pd.read_sql_query('select distinct country, province from wine_ratings', con=engine)
loc_info_df

,country,province
0,Bulgaria,Thracian Valley
1,South Africa,Bot River
2,Romania,Murfatlar
3,Spain,Central Spain
4,Austria,Neusiedlersee
...,...,...
420,New Zealand,Canterbury
421,Slovenia,Slovenska Istra
422,Croatia,Dalmatian Coast
423,Greece,Achaia


### Start of Transform
* Add Long and Lat columns to data set

In [10]:
loc_info_df['latitude'] = ""
loc_info_df['longitude'] = ""
loc_info_df.head()

,country,province,latitude,longitude
0,Bulgaria,Thracian Valley,,
1,South Africa,Bot River,,
2,Romania,Murfatlar,,
3,Spain,Central Spain,,
4,Austria,Neusiedlersee,,


* Use Google API to find longitude and latitude for each of the locations in the data set

In [11]:
#Get the lat and long for the API

# Build the endpoint URL
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'

#Need the province , country from the table and combine together
#Create loop to gather the information

print("Beginning Data Retrieval")
print("-----------------------------")
set=1
# Loop through the list of province & country and perform a request for data on each
for index, row in loc_info_df.iterrows():
    
    #Get the fields from the table
    province_name = row['province']
    country_name = row['country']

    location = (province_name) + ", " + country_name

    geo_response = requests.get(geo_url.format(location, g_key)).json()

    try:
        # Extract latitude and longitude

        loc_info_df.loc[index, "latitude"] = geo_response["results"][0]["geometry"]["location"]["lat"]
        loc_info_df.loc[index, "longitude"] = geo_response["results"][0]["geometry"]["location"]["lng"]
        
        if index % 100 == 0:
            print(f"Processing Record {index} of location {province_name}, {country_name}.")
        
    except (KeyError):
        print(f"{province_name}, {country_name} not found for Record {index} . Skipping...")
        set+= 1
    except (IndexError):
        print(f"Location not found has Index error {index} for {province_name}, {country_name}. Skipping...")
        set+= 1


Beginning Data Retrieval
-----------------------------
Processing Record 0 of location Thracian Valley, Bulgaria.
Location not found has Index error 30 for Rheingau, Germany. Skipping...
Location not found has Index error 64 for Pageon, Greece. Skipping...
Location not found has Index error 82 for Haut-Judeé, Israel. Skipping...
Processing Record 100 of location Korčula, Croatia.
Location not found has Index error 111 for Vinho Espumante de Qualidade, Portugal. Skipping...
Location not found has Index error 133 for Agioritikos, Greece. Skipping...
Location not found has Index error 134 for Thrace, Turkey. Skipping...
Location not found has Index error 180 for Pitsilia Mountains, Cyprus. Skipping...
Location not found has Index error 191 for Vinho Licoroso, Portugal. Skipping...
Processing Record 200 of location Negev Hills, Israel.
Location not found has Index error 201 for Retsina, Greece. Skipping...
Location not found has Index error 202 for Dealurile Munteniei, Romania. Skipping...

In [12]:
loc_info_df.head()

,country,province,latitude,longitude
0,Bulgaria,Thracian Valley,42.144,24.7484
1,South Africa,Bot River,-34.2254,19.1969
2,Romania,Murfatlar,44.1736,28.4105
3,Spain,Central Spain,40.4637,-3.74922
4,Austria,Neusiedlersee,47.865,16.7776


* Replace data where longitude or latitude data was not retreived with NaN so that dropna can remove the data

In [16]:
#Remove data when the longitude or latitude data is blank
loc_info_df['latitude'].replace('', np.nan, inplace=True)
loc_info_df['longitude'].replace('', np.nan, inplace=True)
loc_info_df = loc_info_df.dropna(subset=['latitude', 'longitude'])

### Load Process 
* Load country, province, long, and lat data to PostGresSQL location_info table

In [17]:
loc_info_df.to_sql(name='location_info', con=engine, if_exists='append', index=False)

In [18]:
pd.read_sql_query('select * from location_info', con=engine).head()

,country,province,latitude,longitude
0,Bulgaria,Thracian Valley,42.144026,24.748362
1,South Africa,Bot River,-34.225425,19.196924
2,Romania,Murfatlar,44.173552,28.410548
3,Spain,Central Spain,40.463667,-3.749220
4,Austria,Neusiedlersee,47.865043,16.777559
